In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002EE65BB3BF0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002EE65BDD160>, root_client=<openai.OpenAI object at 0x000002EE63A986B0>, root_async_client=<openai.AsyncOpenAI object at 0x000002EE65BB3C20>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [13]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose=True)

E:\python\mcq_generator\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [17]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain=SequentialChain(chains=[review_chain, quiz_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [19]:
file_path = "E:/python/mcq_generator/data.txt"

In [20]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [22]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [23]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER=5,
SUBJECT="biology",
TONE="simple"

In [26]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

E:\python\mcq_generator\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1921
Prompt Tokens:1202
Completion Tokens:719
Total Cost:0.0016795


In [28]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [31]:
quiz=response.get("quiz")
quiz=json.loads(quiz)

In [32]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'What is the main theme that explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Revolution',
   'c': 'Mutation',
   'd': 'Adaptation'},
  'correct': 'a'},
 '3': {'mcq': 'Which of the following is NOT a major theme in biology?',
  'options': {'a': 'Energy processing',
   'b': 'Evolution',
   'c': 'Weather patterns',
   'd': 'Regulation of internal environments'},
  'correct': 'c'},
 '4': {'mcq': 'What is the process that allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Energy processing',
   'd': 'Digestion'},
  'correct': 'c'},
 '5': {'mcq': 'Which of the following is NOT a eukaryotic organism?',
  'options': {'a': 'Protists', 'b': 'Bacteria', 'c': 'Plants', 'd': 'Animals'},
  'correct': 'b'}}

In [33]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [34]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'What is the main theme that explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Revolution | c: Mutation | d: Adaptation',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a major theme in biology?',
  'Choices': 'a: Energy processing | b: Evolution | c: Weather patterns | d: Regulation of internal environments',
  'Correct': 'c'},
 {'MCQ': 'What is the process that allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Energy processing | d: Digestion',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a eukaryotic organism?',
  'Choices': 'a: Protists | b: Bacteria | c: Plants | d: Animals',
  'Correct': 'b'}]

In [35]:
quiz = pd.DataFrame(quiz_table_data)

In [36]:
quiz.to_csv("quiz.csv",index=False)